In [ ]:
!pip install langchain mistral langchainhub faiss-cpu

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.2/363.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
import os

# Path to the dataset folder
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Langchain_Assignment/langchain-assignment-dataset-main/langchain-assignment-dataset-main/stories'

# List all story files
story_files = [os.path.join(dataset_path, file) for file in os.listdir(dataset_path) if file.endswith('.txt')]

print(f"Found {len(story_files)} story files.")


Found 5 story files.


In [10]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [17]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.4 MB/s eta 0:00:00


In [19]:
!pip install sentence-transformers


In [13]:
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "API-key"


In [15]:
from langchain.embeddings import OpenAIEmbeddings

# Initialize embeddings model
embeddings = OpenAIEmbeddings()
print("Embeddings model initialized successfully!")


Embeddings model initialized successfully!


In [29]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import pickle

# Initialize a local embeddings model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and efficient

def compute_embeddings_local(story_files):
    documents = []
    metadata = []
    embeddings_list = []

    # Process each story file
    for story_file in story_files:
        with open(story_file, 'r') as file:
            content = file.read()
            documents.append(content)
            metadata.append({'story_title': os.path.basename(story_file)})
            embeddings_list.append(model.encode(content))

    # Convert embeddings list to a NumPy array
    embeddings = np.array(embeddings_list)

    # Create a FAISS index
    dimension = embeddings.shape[1]  # Dimensionality of embeddings
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    # Save the FAISS index and metadata
    faiss.write_index(index, "faiss_index.bin")
    with open("metadata.pkl", "wb") as f:
        pickle.dump(metadata, f)

    print("Local embeddings computed and stored!")

compute_embeddings_local(story_files)


Local embeddings computed and stored!


In [30]:
def get_character_info_local(character_name):
    # Load the FAISS index and metadata
    index = faiss.read_index("faiss_index.bin")
    with open("metadata.pkl", "rb") as f:
        metadata = pickle.load(f)

    # Encode the query
    query_embedding = model.encode(character_name).reshape(1, -1)

    # Search for similar embeddings
    _, indices = index.search(query_embedding, k=3)

    if len(indices[0]) == 0:
        return {"error": "Character not found in the stories."}

    # Extract details of the most relevant match
    closest_story_index = indices[0][0]
    story_metadata = metadata[closest_story_index]
    story_title = story_metadata['story_title']

    # Return structured information
    return {
        "name": character_name,
        "storyTitle": story_title,
        "summary": f"A summary of '{character_name}' in the story.",
        "relations": [],
        "characterType": "Unknown"
    }

# Example usage
character_name = "Jon Snow"
character_info = get_character_info_local(character_name)
print(character_info)


{'name': 'Jon Snow', 'storyTitle': 'sorrow.txt', 'summary': "A summary of 'Jon Snow' in the story.", 'relations': [], 'characterType': 'Unknown'}


In [31]:
# Example: Query a character name
character_name = "Jon Snow"  # Replace with any character's name
character_info = get_character_info_local(character_name)
print(character_info)


{'name': 'Jon Snow', 'storyTitle': 'sorrow.txt', 'summary': "A summary of 'Jon Snow' in the story.", 'relations': [], 'characterType': 'Unknown'}
